In [83]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from pymongo import MongoClient
client = MongoClient()
import psycopg2
import csv
import pandas as pd
import json
import sys
from psql_methods import create_empty_psql_tables_from_mongo, fill_psql_from_csv, fields_to_string

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# Code to create the list of fields per collection
# def get_collection_keys(collection):
#     cursor = collection.find({})
#     return list(set([key for document in cursor for key in document.keys()]))
# db_collection = {db['name']: {
#     collection_name: get_collection_keys(client[db['name']][collection_name]) 
#     for collection_name in client[db['name']].list_collection_names()} 
#                  for db in client.list_databases() if db['name'] not in ['admin', 'config', 'local']}
with open('mongo_csvs/db_collection.json', 'r') as f:
    db_collection = json.load(f)
db_collection = db_collection['NFTDB']
del db_collection['testdb']
del db_collection['salesCollection']
del db_collection['slugCollection']
db_collection

{'BotAddress': ['token_id',
  '_id',
  'timestamp',
  'address',
  'transaction_id',
  'slug'],
 'salesCollectionV2': ['token_id',
  '_id',
  'buyer_address',
  'usd_price',
  'is_bundle',
  'payment_token',
  'total_price',
  'timestamp',
  'bundle_token_ids',
  'buyer_username',
  'seller_username',
  'slug',
  'seller_address',
  'event_type',
  'auction_type'],
 'completedSlugsTransfers': ['_id'],
 'completedSlugs': ['_id'],
 'NFTStats': ['one_hour_change',
  'one_minute_sales',
  'six_hour_volume',
  'six_hour_difference',
  'five_minute_average_price',
  'one_minute_sales_change',
  'five_minute_sales_change',
  'six_hour_sales',
  'fifteen_minute_sales',
  'total_volume',
  'six_hour_sales_change',
  'one_minute_volume',
  'seven_day_volume',
  'fifteen_minute_change',
  'one_minute_difference',
  'thirty_day_volume',
  'thirty_day_average_price',
  'one_minute_average_price',
  'six_hour_change',
  'fifteen_minute_difference',
  'num_reports',
  'seven_day_change',
  'total_sal

In [79]:
db_collection_types = {}
for collection_name in db_collection:
    temp_df = pd.read_csv(f'mongo_csvs/{collection_name}.csv', escapechar='\\')
    field_types = {}
    for field in temp_df:
        if len(temp_df.dropna(subset=[field])) < 1:
            field_types[field] = None
        else:
            value = temp_df.dropna(subset=[field]).iloc[0][field]
            if type(value) != str:
                field_types[field] = str(type(value))
            elif value.startswith('0x'):
                field_types[field] = 'BYTEA'
            # this needs to come before timestamp
            elif value.startswith('{') and ':' in value:
                field_types[field] = 'json'
            elif value.startswith('{'):
                field_types[field] = 'ARRAY'
            elif 'timestamp' in value or 'date' in value:
                field_types[field] = 'TIMESTAMP'
            else:
                field_types[field] = str(str)
    db_collection_types[collection_name] = field_types

/tmp/ipykernel_9370/3550739026.py:3: DtypeWarning: Columns (0,14) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'mongo_csvs/{collection_name}.csv', escapechar='\\')
/tmp/ipykernel_9370/3550739026.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'mongo_csvs/{collection_name}.csv', escapechar='\\')
/tmp/ipykernel_9370/3550739026.py:3: DtypeWarning: Columns (0,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'mongo_csvs/{collection_name}.csv', escapechar='\\')
/tmp/ipykernel_9370/3550739026.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'mongo_csvs/{collection_name}.csv', escapechar='\\')


In [55]:
# this constructs a new db from salesCollection with consistent types per field
# 

# sales_df = pd.DataFrame(list((client.NFTDB.salesCollection.find({}))))
# sales_df['bundle_token_ids'] = sales_df['token_id'].apply(lambda x: x if type(x) == list else None)
# sales_df['token_id'] = sales_df['token_id'].apply(lambda x: x if type(x) != list else None)
# client.NFTDB.salesCollectionV2.insert_many(sales_df.to_dict('records'))

In [55]:
def mongo2csv(csv_columns, collection, collection_name):
    with open(f'mongo_csvs/{collection_name}.csv', 'w', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns, escapechar='\\', quoting=csv.QUOTE_NONE)
        writer.writeheader()
        for doc in collection.find({}):
            edited_doc = {}
            for key in doc:
                
                # write as hex
                try:
                    if type(doc[key]) == str and (doc[key].startswith(r'\x')):
                        edited_doc[key] = doc[key].replace(r'\x', '0x')
                        # edited_doc[key] = bytes.fromhex(doc[key][2:])
                        # edited_doc[key] = doc[key].replace('0x', r'\x')
                    else:
                        edited_doc[key] = doc[key]
                except AttributeError:
                    print(collection_name, doc)
                except ValueError as error:
                    print(error, doc[key][2:], )
                
            # This needs to run after the first block
            # opensea_fees and seller_fees to dict
            if collection_name == 'listingsCollection':
                # print(edited_doc)
                if edited_doc['opensea_fees']:
                    edited_doc['opensea_fees'] = {pair[0]: pair[1] for pair in edited_doc['opensea_fees']}
                if edited_doc['seller_fees']:
                    edited_doc['seller_fees'] = {pair[0]: pair[1] for pair in edited_doc['seller_fees']}

            for key in edited_doc:
                # this needs to run after the opensea block
                if type(edited_doc[key]) == list:
                    # print(doc[key])
                    edited_doc[key] = '{' + ','.join(edited_doc[key]) + '}'
            writer.writerow(edited_doc)

for collection_name in db_collection:
    if collection_name != 'testdb' or collection_name != 'slugCollection' or collection_name != 'salesCollection':
        mongo2csv(db_collection[collection_name], client['NFTDB'][collection_name], collection_name)
    

In [57]:
ba_df = pd.read_csv('mongo_csvs/BotAddress.csv')
ba_df = ba_df[ba_df['token_id'].isnull()]
ba_df.to_csv('mongo_csvs/BotAddress.csv', index = False)

In [ ]:
stats_df = pd.read_csv('mongo_csvs/NFTStats.csv')
stats_df
# ba_df = ba_df[ba_df['token_id'].isnull()]
# stats_df.to_csv('mongo_csvs/BotAddress.csv', index = False)

In [63]:
fields_to_string(db_collection_types, 'BotAddress')

'BotAddress None, salesCollectionV2 None, completedSlugsTransfers None, completedSlugs None, NFTStats None, nftCombinations None, completedSlugsHistoric None, completedSlugsListings None, addresstoNFT None, listingsCollection None, transferCollection None, address_data None, addressCollection None'

In [124]:
create_empty_psql_tables_from_mongo(db_collection_types)

In [125]:
for collection_name in db_collection_types:
    fill_psql_from_csv(collection_name)

In [126]:
with open('mongo_csvs/db_collection_types', 'w') as f:
    json.dump(db_collection_types, f)